- Kombiner alle transcripts
- Fjern alle taler under 30 sec
- Find unikke politikere og deres samlede taletid
- Fjern alle der taler mindre end 2 min
- Tilføj køn
- Vælg 20 random kvinder og mænd
- I et loop over de valgte politikere:
    - I DF der er sorteret for at indeholde mindst 30 sec taletid
    - Filtrer på den valgte politiker
    - Sample én tale ad gangen, indtil der er mindst 120 sec taletid
    - Føj til en slice df


-audiopreprocess-



In [1]:
import pandas as pd
import glob
import os
from moviepy.editor import VideoFileClip
from tqdm import tqdm

In [2]:
def load_all_csv_to_df(directory_path):
    # Create a pattern for glob to match all CSV files in the directory
    csv_pattern = f"{directory_path}/*.csv"
    
    # Use glob to get a list of CSV files in the directory
    csv_files = glob.glob(csv_pattern)
    
    # Read each CSV file into a DataFrame and store them in a list
    df_list = [pd.read_csv(file) for file in csv_files]

    # Concatenate all DataFrames in the list into a single DataFrame
    combined_df = pd.concat(df_list, ignore_index=True)

    return combined_df

In [3]:
def trim_video(video_file_path, start_time, end_time, trimmed_video_path):
    video = VideoFileClip(video_file_path).subclip(start_time, end_time)
    video.write_videofile(trimmed_video_path, codec="libx264", audio_codec="aac")

In [4]:
directory_path = "../data/transcripts"
all_transcripts = load_all_csv_to_df(directory_path)

In [5]:
# filter and remove where duration is under 30
all_transcripts = all_transcripts[all_transcripts['duration'] > 30]
all_transcripts = all_transcripts[all_transcripts['duration'] < 200]

In [6]:
speakers_time = all_transcripts.groupby('name')['duration'].sum().reset_index()
speakers_time.columns = ['name', 'cum_duration']

In [7]:
# filter out speakers with less than 120 seconds of speaking time
speakers_time = speakers_time[speakers_time['cum_duration'] > 120]

In [8]:
politicians = pd.read_csv('../data/politicians.csv')

In [9]:
# add a single column (gender) from politicians to speakers_time by joining on name

speakers_time = speakers_time.merge(politicians[['name', 'gender']], on='name', how='left')

In [10]:
# find unique values in gender column
speakers_time['gender'].value_counts()

gender
f    24
m    23
Name: count, dtype: int64

In [11]:
# filter speakers_time to have one with males and one with females
female = speakers_time[speakers_time['gender'] == 'f']
female = female['name'] # keep only name column
female = female.sample(n=20, random_state=1)  # 'random_state' for reproducibility

male = speakers_time[speakers_time['gender'] == 'm']
male = male['name'] # keep only name column
male = male.sample(n=20, random_state=1)  # 'random_state' for reproducibility

# combine male and female into one list
politician_sample = pd.concat([female, male])
#drop the index
politician_sample = politician_sample.reset_index(drop=True)



In [12]:
import pandas as pd

# Initialize an empty DataFrame for the result
sample_clips = pd.DataFrame()

# Loop over each politician in the politician_sample
for politician in politician_sample:
    # Filter all_transcripts for the current politician
    politician_transcripts = all_transcripts[all_transcripts['name'] == politician]
    
    # Initialize a DataFrame for accumulating transcripts for the current politician
    accumulated_transcripts = pd.DataFrame()

    # Loop until the sum of duration exceeds 120 seconds
    while accumulated_transcripts.empty or accumulated_transcripts['duration'].sum() < 120:
        # Sample a random row
        random_row = politician_transcripts.sample(n=1)
        # Check if the random row has already been added
        if random_row['start_time'].values[0] in accumulated_transcripts['start_time'].values:
            continue
        # Append the random row to the accumulated_transcripts DataFrame
        accumulated_transcripts = pd.concat([accumulated_transcripts, random_row])

        # Check if the accumulated transcripts exceed the desired duration
        if accumulated_transcripts['duration'].sum() >= 120:
            break

    # Append the accumulated_transcripts for this politician to the result_df
    sample_clips = pd.concat([sample_clips, accumulated_transcripts])


In [23]:
#save the sample_clips to a csv, semi-colon separated, encoding
sample_clips.to_csv('../data/sample_clips.csv', sep=';', encoding='utf-8', index=False)

In [13]:
sample_clips.groupby('name')['duration'].sum().reset_index()

,name,duration
0,Alexander Ryle,213.0
1,Anders Kronborg,152.0
2,Anne Paulin,245.0
3,Benny Engelbrecht,160.0
4,Betina Kastbjerg,148.0
5,Birgitte Vind,127.0
6,Brigitte Klintskov Jerkel,122.0
7,Charlotte Bagge Hansen,165.0
8,Erling Bonnesen,140.0
9,Hans Andersen,135.0


VIDEO slicing and audio saving

In [14]:
#subset the sample_clips to the first 5 rows
tester = sample_clips.iloc[:5]

In [15]:
# create a folder within ../data/raw_videos/politicians/ for each politician
for politician in politician_sample:
    os.makedirs(f"../data/politicians/{politician}", exist_ok=True)

In [16]:
#loop over each row in sample_clips
for index, row in tqdm(sample_clips.iterrows()):
    # Replace colon in 'start_min' with an underscore for the filename
    start_min_formatted = row['start_min'].replace(':', '-')

    # Create a filename for saving the trimmed video
    save_path = f"../data/politicians/{row['name']}/video_{row['video_id']}_start_{start_min_formatted}.mp4"

    # if the file already exists, skip it
    if os.path.exists(save_path):
        continue

    #create a path to the original video
    video_file_path = f"../data/raw_videos/{row['video_id']}.mp4"

    #trim the video
    trim_video(video_file_path, row['start_sec'], row['end_sec'], save_path)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Moviepy - Building video ../data/politicians/Marlene Harpsøe/video_38_start_76-41.mp4.
MoviePy - Writing audio in video_38_start_76-41TEMP_MPY_wvf_snd.mp4


0it [00:03, ?it/s]

MoviePy - Done.
Moviepy - Writing video ../data/politicians/Marlene Harpsøe/video_38_start_76-41.mp4



...

Moviepy - Building video ../data/politicians/Kaare Dybvad Bek/video_41_start_62-09.mp4.
MoviePy - Writing audio in video_41_start_62-09TEMP_MPY_wvf_snd.mp4


98it [08:41,  3.98s/it]

MoviePy - Done.
Moviepy - Writing video ../data/politicians/Kaare Dybvad Bek/video_41_start_62-09.mp4



99it [08:44,  5.30s/it]

Moviepy - Done !
Moviepy - video ready ../data/politicians/Kaare Dybvad Bek/video_41_start_62-09.mp4
